# SCC0270 - Redes Neurais e Aprendizado Profundo
### Aula 8 - Prática (Redes neurais convolutivas)
**Daniel Penna Chaves Bertazzo - 10349561**

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard

# Item 1.1 - Preparação dos dados

###  Importação do *dataset* e concatenação em um único conjunto

In [2]:
# Carrega o dataset
# originalmente ja vem separado em treino e teste, mas aqui vamos juntar tudo
(data, target), (data_aux, target_aux) = tf.keras.datasets.mnist.load_data()

In [3]:
# Junta as instancias dos dois conjuntos em um unico dataset
data = np.concatenate((data, data_aux))

# Junta os targets dos dois conjuntos em um unico vetor
target = np.concatenate((target, target_aux))

# Para economizar memoria
del data_aux, target_aux

In [4]:
# Faz o one-hot enconding dos targets (compatibilidade com a saida de uma rede neural)
target = to_categorical(target)

In [5]:
print(data.shape, target.shape, sep='\n')

(70000, 28, 28)
(70000, 10)


### Definição das proporções dos cortes

In [6]:
# Calcula o tamanho de cada corte -> 10% até 100%
cuts = [int((x/10) * data.shape[0]) for x in range(1, 11)]

### Realização dos cortes

In [7]:
X = [] # Armazena os 10 conjuntos de instancias
y = [] # Armazena os 10 conjuntos de targets

In [8]:
for cut in cuts:
    X.append(data[:cut])
    y.append(target[:cut])

### Separação em treino e teste

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
# Armazenam as instancias para treino e teste
X_train = [None] * 10
X_test  = [None] * 10

# Armazenam os targets para treino e teste
y_train = [None] * 10
y_test  = [None] * 10

In [11]:
for i in range(10):
    X_train[i], X_test[i], y_train[i], y_test[i] = train_test_split(X[i], y[i], test_size=0.3)

### Ajustando as dimensões para ser compatível com o modelo do Keras

In [12]:
for i in range(10):
    X_train[i] = X_train[i].reshape(X_train[i].shape[0], X_train[i].shape[1], X_train[i].shape[2], 1)
    X_test[i]  = X_test[i].reshape(X_test[i].shape[0], X_test[i].shape[1], X_test[i].shape[2], 1)

#  Item 1.2 - Criação e implementação do modelo
A arquitetura utilizada para todos os cortes será a seguinte:
* **Primeira camada:** convolução com 32 filtros 3x3 e ativação relu + max_pooling 2x2

* **Segunda camada:** convolução com 64 filtros 3x3 e ativação relu + max_pooling 2x2

* **Última camada:** *fully connected* com 10 neurônios (número de classes) e ativação softmax

In [13]:
model = Sequential()

# Primeira camada (conv + pooling)
# Na primeira camada, é preciso passar o input_shape. Como todos os X_train[i] 
# possuem dados da mesma dimensao, divergindo apenas na quantidade de instancias em cada um,
# pode-se usar o shape de qualquer um deles. Neste caso, usou-se o X_train[0]
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=X_train[0].shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Segunda camada (conv + pooling)
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=X_train[0].shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Transforma os dados para um vetor unidimensional
model.add(Flatten())

# Ultima camada: fully connected com 10 neuronios (1 para cada classe) e
# ativacao softmax para obter as probabilidades
model.add(Dense(10, activation='softmax'))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                16010     
Total params: 34,826
Trainable params: 34,826
Non-trainable params: 0
____________________________________________________

In [16]:
tensorboards = []
for i in range(10):
    name = "mnist_cnn_" + str((i+1)*10)
    
    tensorboards.append(TensorBoard(log_dir="logs/{}".format(name)))

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
# Treina e testa (valida) o modelo para todos os cortes do dataset
for i in range(10):
    print("============== MODELO %d ==============" % i)
    model.fit(X_train[i],
              y_train[i],
              epochs=3,
              verbose=2,
              validation_data=(X_test[i], y_test[i]),
              callbacks=[tensorboards[i]])

============== MODELO 0 ==============
Train on 4900 samples, validate on 2100 samples
Epoch 1/3
4900/4900 - 3s - loss: 3.2724 - accuracy: 0.8086 - val_loss: 0.5138 - val_accuracy: 0.9105
Epoch 2/3
4900/4900 - 2s - loss: 0.2356 - accuracy: 0.9518 - val_loss: 0.4013 - val_accuracy: 0.9214
Epoch 3/3
4900/4900 - 3s - loss: 0.1813 - accuracy: 0.9598 - val_loss: 0.3347 - val_accuracy: 0.9343
============== MODELO 1 ==============
Train on 9800 samples, validate on 4200 samples
Epoch 1/3
9800/9800 - 5s - loss: 0.1999 - accuracy: 0.9541 - val_loss: 0.1653 - val_accuracy: 0.9569
Epoch 2/3
9800/9800 - 4s - loss: 0.0765 - accuracy: 0.9798 - val_loss: 0.1821 - val_accuracy: 0.9574
Epoch 3/3
9800/9800 - 5s - loss: 0.0458 - accuracy: 0.9860 - val_loss: 0.1705 - val_accuracy: 0.9624
============== MODELO 2 ==============
Train on 14700 samples, validate on 6300 samples
Epoch 1/3
14700/14700 - 7s - loss: 0.1106 - accuracy: 0.9725 - val_loss: 0.0905 - val_accuracy: 0.9760
Epoch 2/3
14700/14700 - 8s - 

## Resultados e gráficos
Nesta seção, serão mostradas imagens de dados gerados com a ferramenta *tensorboard*, acionada no momento de treinamento do modelo. Essa decisão foi realizada com o objetivo de facilitar o acesso a tais dados, colocando-os no mesmo arquivo que o resto da atividade (este *notebook*). Porém, é possível acessar as informações e gráficos e interagir com eles por meio do seguinte passo a passo:
* Abra um terminal no mesmo diretório dos arquivos deste trabalho
* Digite o comando `tensorboard --logdir=logs/.`. Algo parecido com isso aparecerá:

![response](img/response.png)

* Em seguida, abra o *link* retornado pelo comando (`http://localhost:6006`).

Com isso, os arquivos de relatório gerados pela ferramenta, presentes na pasta `logs` será aberto em uma aba no navegador, possibilitando visualizações melhores e interativas.

As imagens conterão um gráfico com a evolução das acurácias do conjunto de treino e validação ao longo das 3 iterações *(epochs)*, assim como uma legenda contendo, em detalhes, os valores dessas métricas.

# Corte de 10%

## *Epoch* 1

![epoch1](img/model10_1.png)

## *Epoch* 2

![epoch2](img/model10_2.png)

##  *Epoch* 3

![epoch3](img/model10_3.png)

# Corte de 20%

## *Epoch* 1

![epoch1](img/model20_1.png)

## *Epoch* 2

![epoch2](img/model20_2.png)

##  *Epoch* 3

![epoch3](img/model20_3.png)